# Average Articulation 
computed an average articulation of a disyllabic vowel utterances
- Created: 02 April 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import librosa
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import librosa.display
from matplotlib import cm
from functools import partial

import sklearn
from sklearn.decomposition import PCA
from os import makedirs
import os
from os.path import join
import shutil
from IPython import display

%matplotlib inline

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# np.random.seed(42)

In [3]:
EXP = '31_2'
# data_path = '../data/d_eval'
data_path = '../data/d_records/record_all'

In [4]:
# path = '../experiment_2/result/eval_{}/testset.npz'.format(EXP)
path = '../experiment/result/predict_{}/testset.npz'.format(EXP)
data = np.load(path)['param_sets']
data = data.reshape((data.shape[0]*data.shape[1],24))
df = pd.DataFrame(data)

In [5]:
with open(join(data_path,'syllable_name.txt')) as f:
    syllable_name = np.array([word.strip() for line in f for word in line.split(',')])
    syllable_name = np.array([ '%s;%s'%(item,str(idx+1)) for pair in syllable_name for idx, item in enumerate(pair)])
    
df['phonetic'] = syllable_name
df['phonetic'], df['position'] = df['phonetic'].str.split(';', 1).str
df = df.groupby(['phonetic','position']).mean().reset_index()
df_1 = df[df['position']=='1'] 
df_2 = df[df['position']=='2'] 

In [6]:
df.head()

,phonetic,position,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,7,1,0.382742,-4.994272,0.0,-4.768025,0.441548,0.482527,0.525424,-0.060628,...,-0.328177,-6.632993,-2.520878,0.502778,0.346976,0.676146,0.161284,-0.05,-0.05,-0.05
1,7,2,0.411095,-4.972227,0.0,-4.655173,0.440525,0.487167,0.541590,-0.062604,...,-0.291734,-6.569276,-2.432999,0.533274,0.342819,0.628186,0.144525,-0.05,-0.05,-0.05
2,E,1,0.302313,-4.239133,0.0,-4.377287,0.013557,0.884552,0.819400,0.012718,...,0.137112,-6.478843,-2.116651,0.961207,0.199938,0.371898,-0.233089,-0.05,-0.05,-0.05
3,E,2,0.278640,-4.248021,0.0,-4.353285,0.012579,0.888473,0.812869,0.011059,...,0.115243,-6.496690,-2.114658,0.967274,0.231580,0.385732,-0.207643,-0.05,-0.05,-0.05
4,M,1,0.561102,-5.092115,0.0,-4.274819,0.560504,0.396843,0.396947,-0.060824,...,0.253738,-6.085338,-1.755572,0.945915,0.303271,0.328356,0.050232,-0.05,-0.05,-0.05


## Create Speaker File

In [7]:
def create_speaker_file(df, file_path):
                        
    speaker_header_file = '../experiment/lib/templates/speaker_template_head.txt'
    speaker_tail_file = '../experiment/lib/templates/speaker_template_tail.txt'

    # Read speaker template
    speaker_head = open(speaker_header_file,'r').read()
    speaker_tail = open(speaker_tail_file,'r').read()

    # load vocaltract param name
    param_name = np.load('../experiment/lib/templates/speaker_param.npz')['name']
    
    pho = set(df['phonetic'])
    
    with open(file_path, 'w') as f:
        f.write(speaker_head)

        for p in pho:
            df_temp = df[df['phonetic']==p].copy()
            f.write('<shape name="{}">\n'.format(p))
            for i in range(24):
                f.write('<param name="{}" value="{:.2f}"/>\n'.format(param_name[i],df_temp[i].values[0]))
            f.write('</shape>\n')

        f.write(speaker_tail)

In [8]:
create_speaker_file(df_1, 'speaker_31_pred_1.speaker')
create_speaker_file(df_2, 'speaker_31_pred_2.speaker')